In [16]:
import pickle
import json
import os
from sklearn.model_selection import train_test_split
from collections import Counter

In [6]:
obs = None
with open('obs.json') as f:
    obs = json.load(f)

res = None
with open('res.json') as f:
    res = json.load(f)